In [1]:
!pip install autogluon

  Obtaining dependency information for autogluon from https://files.pythonhosted.org/packages/2e/42/f26592ecf3dc4e4edcbc5f7f7e3deba25b0681fad2ad04a82d12af17e3a4/autogluon-0.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.core[all]==0.8.2 from https://files.pythonhosted.org/packages/e0/56/545adb1d388e78591cd7e36de0c8b889c1944de362bdaeec0f31d01890df/autogluon.core-0.8.2-py3-none-any.whl.metadata
^C
ERROR: Operation cancelled by user


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import pandas as pd
import numpy as np
import os
import random
from tqdm.auto import tqdm
# import optuna

# DATA_PATH = "/content/drive/MyDrive/JEJU/data/"
DATA_PATH = "/kaggle/input/jeju-dataset/"

# DATA_PATH = '/content/drive/MyDrive/데이터 분석/데이콘 캐글 컴페티션/예슬 개인 참가/제주 특산물 가격 예측/data/'
# DATA_PATH = "C:\python-code\DACON_JEJU\data/"
SEED = 42

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED) # Seed 고정

In [5]:
import numpy as snp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn import tree
# from catboost import CatBoostRegressor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from sklearn.model_selection import KFold, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, make_scorer

KeyboardInterrupt: 

```
item : 품목 코드
TG : 감귤
BC : 브로콜리
RD : 무
CR : 당근
CB : 양배추
corporation : 유통 법인 코드
법인 A부터 F 존재
location : 지역 코드
J : 제주도 제주시
S : 제주도 서귀포시
supply(kg) : 유통된 물량, kg 단위
price(원/kg) : 유통된 품목들의 kg 마다의 가격, 원 단위
```

# DATA Load

In [ ]:
train_df = pd.read_csv(f"{DATA_PATH}train.csv")
test_df = pd.read_csv(f"{DATA_PATH}test.csv")
international = pd.read_csv(f"{DATA_PATH}international_trade.csv")
submission = pd.read_csv(f"{DATA_PATH}sample_submission.csv")

In [ ]:
train_df.rename(columns={'supply(kg)':'supply', 'price(원/kg)':'price'}, inplace=True)

# Preprocessing

In [ ]:
# 공휴일
holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']

In [ ]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
train_df['year'] = train_df['timestamp'].dt.year
train_df['month'] = train_df['timestamp'].dt.month
train_df['week'] = train_df['timestamp'].dt.isocalendar().week
train_df['day'] = train_df['timestamp'].dt.day
train_df['day_of_year'] = train_df['timestamp'].dt.day_of_year
train_df['day_of_week'] = train_df['timestamp'].dt.dayofweek
# train_df['sin_week'] = np.sin(2 * np.pi * train_df['week'] / 4)
# train_df['cos_week'] = np.cos(2 * np.pi * train_df['week'] / 4)
# train_df['sin_dayofweek'] = np.sin(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
# train_df['cos_dayofweek'] = np.cos(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
train_df['holiday'] = np.where((train_df.day_of_week >= 5) | (train_df.timestamp.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
test_df['year'] = test_df['timestamp'].dt.year
test_df['month'] = test_df['timestamp'].dt.month
test_df['week'] = test_df['timestamp'].dt.isocalendar().week
test_df['day'] = test_df['timestamp'].dt.day
test_df['day_of_year'] = test_df['timestamp'].dt.day_of_year
test_df['day_of_week'] = test_df['timestamp'].dt.dayofweek
# test_df['sin_week'] = np.sin(2 * np.pi * test_df['week'] / 4)
# test_df['cos_week'] = np.cos(2 * np.pi * test_df['week'] / 4)
# test_df['sin_dayofweek'] = np.sin(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
# test_df['cos_dayofweek'] = np.cos(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
test_df['holiday'] = np.where((test_df.day_of_week >= 5) | (test_df.timestamp.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

In [ ]:
import math

# 연중일 주기
def cos_year(date):
    day_of_year = date.timetuple().tm_yday  # 해당 날짜의 연중 몇 번째 날인지를 가져옵니다.
    period = 365  # 1년은 365일
    return math.cos(2 * math.pi * day_of_year / period)

def sin_year(date):
    day_of_year = date.timetuple().tm_yday  # 해당 날짜의 연중 몇 번째 날인지를 가져옵니다.
    period = 365  # 1년은 365일
    return math.sin(2 * math.pi * day_of_year / period)

train_df['cos_year'] = train_df['timestamp'].apply(cos_year)
train_df['sin_year'] = train_df['timestamp'].apply(sin_year)

test_df['cos_year'] = test_df['timestamp'].apply(cos_year)
test_df['sin_year'] = test_df['timestamp'].apply(sin_year)

# 요일 주기 (주간)
def cos_weekday(date):
    day_of_week = date.weekday()  # Monday is 0, Sunday is 6
    period = 7  # 1 week has 7 days
    return math.cos(2 * math.pi * day_of_week / period)

def sin_weekday(date):
    day_of_week = date.weekday()  # Monday is 0, Sunday is 6
    period = 7  # 1 week has 7 days
    return math.sin(2 * math.pi * day_of_week / period)

train_df['cos_dow'] = train_df['timestamp'].apply(cos_weekday)
train_df['sin_dow'] = train_df['timestamp'].apply(sin_weekday)

test_df['cos_dow'] = test_df['timestamp'].apply(cos_weekday)
test_df['sin_dow'] = test_df['timestamp'].apply(sin_weekday)

# 연중주 주기
def cos_week_of_year(date):
    week_of_year = date.isocalendar()[1]  # Get the ISO week number
    period = 52  # 1 year has 52 weeks
    return math.cos(2 * math.pi * week_of_year / period)

def sin_week_of_year(date):
    week_of_year = date.isocalendar()[1]  # Get the ISO week number
    period = 52  # 1 year has 52 weeks
    return math.sin(2 * math.pi * week_of_year / period)

train_df['cos_week_of_year'] = train_df['timestamp'].apply(cos_week_of_year)
train_df['sin_week_of_year'] = train_df['timestamp'].apply(sin_week_of_year)

test_df['cos_week_of_year'] = test_df['timestamp'].apply(cos_week_of_year)
test_df['sin_week_of_year'] = test_df['timestamp'].apply(sin_week_of_year)

## 이동평균 칼럼

In [ ]:
train_df['item_id'] = train_df.ID.str[0:6]
test_df['item_id'] = test_df.ID.str[0:6]

window_size = 7

for item_id in train_df['item_id'].unique():
    df = train_df[train_df['item_id'] == item_id]
    df['price_MA'] = df['price'].rolling(window=window_size).mean()
    df['supply_MA'] = df['supply'].rolling(window=window_size).mean()

    # Update the rolling mean values in the original DataFrame
    train_df.loc[train_df['item_id'] == item_id, 'price_MA'] = df['price_MA']
    train_df.loc[train_df['item_id'] == item_id, 'supply_MA'] = df['supply_MA']


In [ ]:
train_df.isna().sum()

In [ ]:
# window_size = 7

# train_df['price_MA'] = train_df['price'].rolling(window=window_size).mean()
# # test_df['price_MA'] = test_df['price'].rolling(window=window_size).mean()

# train_df['supply_MA'] = train_df['supply'].rolling(window=window_size).mean()
# # test_df['supply_MA'] = test_df['supply'].rolling(window=window_size).mean()

# train_df['price_MA'].interpolate(method='linear', inplace=True)
# train_df['supply_MA'].interpolate(method='linear', inplace=True)


## 월별 품목의 가격 추가(mean, max)

In [ ]:
train_df['month_of_year'] = train_df['timestamp'].dt.to_period('M')
test_df['month_of_year'] = test_df['timestamp'].dt.to_period('M')


In [ ]:
item_month_price = train_df.groupby(['item', 'month_of_year'])
agg = {'price' : ['mean', 'max']}
price_df = item_month_price.agg(agg).reset_index()

In [ ]:
price_df.columns = ['item', 'month_of_year', 'month_price_mean', 'month_price_max']
train_df = pd.merge(train_df, price_df, on=['item', 'month_of_year'], how='left')

In [ ]:
# 테스트셋 반영
test_df = pd.merge(test_df, price_df, on=['item', 'month_of_year'], how='left')

In [ ]:
train_df

## 월별 품목의 물량 추가(mean, max)

In [ ]:
item_month_supply = train_df.groupby(['item', 'month_of_year'])
agg = {'supply' : ['mean', 'max']}
supply_df = item_month_supply.agg(agg).reset_index()


In [ ]:
supply_df.columns = ['item', 'month_of_year', 'month_supply_mean', 'month_supply_max']
train_df = pd.merge(train_df, supply_df, on=['item', 'month_of_year'], how='left')
train_df.head()

In [ ]:
# 테스트셋 반영
test_df = pd.merge(test_df, supply_df, on=['item', 'month_of_year'], how='left')
test_df.head()

In [ ]:
# 공급과 가격의 상관관계
sup_price_corr = train_df['supply'].corr(train_df['price'])
sup_price_corr

## 유닉스 타임스탬프
- 1970년 1월 1일부터의 초 단위 경과 시간으로 변환

In [ ]:
# # timestamp
# train_df['timestamp'] = (train_df['timestamp'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
# test_df['timestamp'] = (test_df['timestamp'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')

# train_df['timestamp']

In [ ]:
# # month
# train_df['month'] = (train_df['month'].dt.to_timestamp() - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
# test_df['month'] = (test_df['month'].dt.to_timestamp() - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')

# test_df['month']

## international 데이터 merge

In [ ]:
international = pd.read_csv(f"{DATA_PATH}international_trade.csv")


In [ ]:
international = international.rename(columns={
    '기간': 'period',
    '품목명': 'item',
    '수출 중량': 'exWeight',
    '수출 금액': 'exValue',
    '수입 중량': 'imWeight',
    '수입 금액': 'imValue',
    '무역수지': 'tradeBalance'
})


In [ ]:
# international['month'] = pd.to_datetime(international['month'], format='%Y-%m')
# international['month'] = (international['month'] - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')

In [ ]:
international['year'] = international['period'].apply(lambda x : int(x[0:4]))
international['month'] = international['period'].apply(lambda x : int(x[5:7]))
international.drop(columns = 'period', inplace=True)
international

In [ ]:
selected_items = international[international['item'].str.contains('감귤|브로콜리|무|당근|양배추')].copy()

selected_items['item'] = selected_items['item'].replace({
    '감귤': 'TG',
    '브로콜리': 'BC',
    '무': 'RD',
    '당근': 'CR',
    '양배추': 'CB'
}, regex = True)

selected_items['item'] = selected_items['item'].str.replace('.*감귤.*', 'TG')
selected_items['item'] = selected_items['item'].str.replace('.*브로콜리.*', 'BC')
selected_items['item'] = selected_items['item'].str.replace('.*무.*', 'RD')
selected_items['item'] = selected_items['item'].str.replace('.*당근.*', 'CR')
selected_items['item'] = selected_items['item'].str.replace('.*양배추.*', 'CB')

selected_items

In [ ]:
selected_items.item.value_counts()

In [ ]:
train_df = train_df.merge(selected_items, on=['year', 'month', 'item'], how='left')
test_df = test_df.merge(selected_items, on=['year', 'month', 'item'], how='left')

train_df

In [ ]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

train_df.isnull().sum().sum() , test_df.isnull().sum().sum()

## 기타 피쳐추가

In [ ]:
# # 품목별 평균 가격
# item_mean_price = train_df.groupby('item')['price'].mean().reset_index()
# item_mean_price.columns = ['item', 'item_mean_price']

# train_df = train_df.merge(item_mean_price, on='item', how='left')
# test_df = test_df.merge(item_mean_price, on='item', how='left')

# # 법인별 평균 가격
# corp_mean_price = train_df.groupby('corporation')['price'].mean().reset_index()
# corp_mean_price.columns = ['corporation', 'corp_mean_price']

# train_df = train_df.merge(corp_mean_price, on='corporation', how='left')
# test_df = test_df.merge(corp_mean_price, on='corporation', how='left')

# # 지역별 평균 가격
# location_mean_price = train_df.groupby('location')['price'].mean().reset_index()
# location_mean_price.columns = ['location', 'location_mean_price']

# train_df = train_df.merge(location_mean_price, on='location', how='left')
# test_df = test_df.merge(location_mean_price, on='location', how='left')

In [ ]:
train_df

In [ ]:
# day of year별 price, supply

# # Price
# price_day_of_year = train_df.groupby(['item_id', 'day_of_year'])['price'].mean().reset_index()
# train_df = train_df.merge(price_day_of_year, on=['item_id', 'day_of_year'], how='left', suffixes=('', '_day_of_year_mean'))
# test_df = test_df.merge(price_day_of_year, on=['item_id', 'day_of_year'], how='left', suffixes=('', '_day_of_year_mean'))

# # Supply
# supply_day_of_year = train_df.groupby(['item_id', 'day_of_year'])['supply'].mean().reset_index()
# train_df = train_df.merge(supply_day_of_year, on=['item_id', 'day_of_year'], how='left', suffixes=('', '_day_of_year_mean'))
# test_df = test_df.merge(supply_day_of_year, on=['item_id', 'day_of_year'], how='left', suffixes=('', '_day_of_year_mean'))


In [ ]:
# 품목별 평균 가격
item_mean_price = train_df.groupby('item')['price_MA'].mean().reset_index()
item_mean_price.columns = ['item', 'item_mean_price']

train_df = train_df.merge(item_mean_price, on='item', how='left')
test_df = test_df.merge(item_mean_price, on='item', how='left')

# 법인별 평균 가격
corp_mean_price = train_df.groupby('corporation')['price_MA'].mean().reset_index()
corp_mean_price.columns = ['corporation', 'corp_mean_price']

train_df = train_df.merge(corp_mean_price, on='corporation', how='left')
test_df = test_df.merge(corp_mean_price, on='corporation', how='left')

# 지역별 평균 가격
location_mean_price = train_df.groupby('location')['price_MA'].mean().reset_index()
location_mean_price.columns = ['location', 'location_mean_price']

train_df = train_df.merge(location_mean_price, on='location', how='left')
test_df = test_df.merge(location_mean_price, on='location', how='left')

In [ ]:
train_df

In [ ]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

train_df.isnull().sum().sum() , test_df.isnull().sum().sum()

In [ ]:
# window_size = 7

# train_df['rolling_mean'] = train_df['price(원/kg)'].rolling(window=window_size).mean()
# train_df['rolling_std'] = train_df['price(원/kg)'].rolling(window=window_size).std()

# test_df['rolling_mean'] = test_df['price(원/kg)'].rolling(window=window_size).mean()
# test_df['rolling_std'] = test_df['price(원/kg)'].rolling(window=window_size).std()

In [ ]:
# lag_period = 7

# train_df['lag_price'] = train_df['price(원/kg)'].shift(lag_period)
# test_df['lag_price'] = test_df['price(원/kg)'].shift(lag_period)

# Encoding

## 원핫

In [ ]:
# # item OneHotEncoding
# train_df = pd.get_dummies(train_df, columns=['item'])
# test_df = pd.get_dummies(test_df, columns=['item'])

## 라벨

In [ ]:
obj_cols = ['item', 'corporation', 'location']

for col in obj_cols:
    enc = LabelEncoder()
    train_df[col]=enc.fit_transform(train_df[col])
    test_df[col]=enc.transform(test_df[col])

train_df.head()

In [ ]:
# # 데이터셋 저장
# train_df.to_csv(f"{DATA_PATH}train_df_pre.csv", index=False)
# test_df.to_csv(f"{DATA_PATH}test_df_pre.csv", index=False)
# train_df.to_csv("train_df_pre.csv", index=False)
# test_df.to_csv("test_df_pre.csv", index=False)

In [ ]:
train_df['week'] = train_df['week'].astype(float)
train_df['cos_week_of_year'] = train_df['cos_week_of_year'].astype(float)
train_df['sin_week_of_year'] = train_df['sin_week_of_year'].astype(float)

test_df['week'] = test_df['week'].astype(float)
test_df['cos_week_of_year'] = test_df['cos_week_of_year'].astype(float)
test_df['sin_week_of_year'] = test_df['sin_week_of_year'].astype(float)

In [ ]:
train_df.info()

In [ ]:
test_df.info()

# AutoGluon

In [ ]:
target = train_df['price']
target

In [ ]:


train_df

In [ ]:
test_df

In [ ]:
train_dropcols = ['ID', 'supply', 'month_of_year','price_MA', 'supply_MA']
test_dropcols = ['ID', 'month_of_year']

In [ ]:
train_ft = train_df.drop(columns = train_dropcols)
test_ft = test_df.drop(columns = test_dropcols)

In [ ]:
train_ft.shape, test_ft.shape

In [ ]:
train_ft['item_id']

In [ ]:
# from autogluon.tabular import TabularPredictor
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
data = TimeSeriesDataFrame(train_ft)

In [ ]:
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price",
    eval_metric="RMSE"
)
# seed 고정
predictor.fit(data, random_seed=SEED)

# 예측

In [ ]:
# 모델 재학습
predictor.refit_full()

In [ ]:
pred = predictor.predict(data, random_seed=SEED)
pred

In [ ]:
pred = pred.reset_index()['mean']
pred

In [ ]:
# 마이너스 값 0으로 치환
pred[pred < 0] = 0

# 제출

In [ ]:
submission

In [ ]:
submission['answer'] = pred
submission

In [ ]:
submission.to_csv(f"{DATA_PATH}submission_ma_1113.csv", index=False)
submission.to_csv("submission_ma_1113.csv", index=False)